# Predicción de Default de Tarjetas de Crédito usando Regresión Logística

Este notebook implementa un modelo de machine learning para predecir el default (incumplimiento) de pagos de tarjetas de crédito utilizando regresión logística con selección de características y optimización de hiperparámetros.

In [1]:
# Librerías fundamentales para análisis de datos y manipulación de archivos
import pandas as pd  # Manipulación de DataFrames y análisis de datos
import os           # Gestión de rutas y operaciones del sistema

## 1. Importación de Librerías y Configuración de Rutas

Configuración inicial del entorno de trabajo con las librerías necesarias y definición de rutas de archivos.

In [2]:
# Configuración de directorios del proyecto
RUTA_DATOS_ENTRADA = "../files/input/"
RUTA_RESULTADOS_SALIDA = "../files/output/"

# Definición de archivos de datos
ARCHIVO_DATASET_ENTRENAMIENTO = os.path.join(RUTA_DATOS_ENTRADA, "train_data.csv.zip")
ARCHIVO_DATASET_VALIDACION = os.path.join(RUTA_DATOS_ENTRADA, "test_data.csv.zip")

## 2. Carga de Datos

Carga de conjuntos de datos de entrenamiento y prueba desde archivos CSV comprimidos.

In [3]:
# Carga del conjunto de datos de entrenamiento
dataframe_entrenamiento_original = pd.read_csv(ARCHIVO_DATASET_ENTRENAMIENTO, compression="zip")
dataframe_entrenamiento_original.sample(5)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
14362,9226,160000,2,2,2,23,0,0,0,-1,...,283,0,-17149,1373,1000,283,0,0,20261,0
18644,7205,370000,2,1,2,28,-1,-1,-1,-1,...,4347,5562,0,63895,0,8694,5562,0,3500,0
1697,19398,240000,2,1,1,33,0,0,0,0,...,77329,50673,48723,4060,3181,3000,1500,3042,1377,0
12011,17760,180000,2,3,2,52,0,0,0,0,...,163447,134939,135815,5902,7693,5464,4893,5095,4862,0
18340,19939,290000,2,2,1,45,0,0,0,0,...,35711,35381,34468,1564,1594,1840,1378,1245,1388,0


In [4]:
# Carga del conjunto de datos de validación
dataframe_validacion_original = pd.read_csv(ARCHIVO_DATASET_VALIDACION, compression="zip")
dataframe_validacion_original.sample(5)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
7981,26665,150000,1,2,2,25,0,0,0,0,...,152357,153133,0,5500,5509,5672,6787,0,832,0
6685,22473,200000,2,2,1,39,3,2,2,2,...,125357,121853,124731,0,6216,10000,0,5000,4552,1
8972,29926,210000,1,1,2,36,0,0,0,0,...,43790,510,20188,5000,11294,13790,1000,20188,16666,0
4363,14610,50000,1,2,2,22,0,0,0,0,...,18852,18020,18537,1342,1500,700,651,1000,1000,1
4468,14988,380000,2,2,2,41,0,0,0,0,...,224029,224022,225978,10018,10000,15000,30000,30251,20000,0


## 3. Preprocesamiento y Limpieza de Datos

Definición de funciones para limpiar y preparar los datos antes del entrenamiento del modelo.

In [5]:
def transformar_datos_credito(dataset_crudo):
    """
    Transforma y limpia datos de tarjetas de crédito para modelado predictivo.
    
    Operaciones realizadas:
    - Creación de variable objetivo con nomenclatura estándar
    - Eliminación de identificadores y columnas redundantes  
    - Filtrado de registros con información completa y válida
    - Normalización de categorías educacionales para reducir dispersión
    
    Args:
        dataset_crudo: DataFrame con datos originales de tarjetas de crédito
        
    Returns:
        DataFrame procesado y listo para entrenamiento de modelos
    """
    # Crear copia independiente del dataset original
    datos_transformados = dataset_crudo.copy()
    
    # Generar variable objetivo con nomenclatura estándar (nombre requerido por tests)
    datos_transformados["default"] = datos_transformados["default payment next month"]
    
    # Remover columnas innecesarias para el análisis predictivo
    datos_transformados = datos_transformados.drop(["ID", "default payment next month"], axis=1)
    
    # Aplicar filtros de calidad de datos
    condicion_datos_validos = (datos_transformados["EDUCATION"] != 0) & (datos_transformados["MARRIAGE"] != 0)
    datos_transformados = datos_transformados[condicion_datos_validos]
    
    # Consolidar categorías educacionales para mejorar generalización
    datos_transformados["EDUCATION"] = datos_transformados["EDUCATION"].apply(
        lambda nivel: nivel if nivel < 4 else 4
    )
    
    return datos_transformados

In [6]:
# Aplicar transformaciones al conjunto de entrenamiento
dataset_entrenamiento_procesado = transformar_datos_credito(dataframe_entrenamiento_original)
dataset_entrenamiento_procesado.sample(5)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
2069,30000,1,1,1,42,0,0,-1,-1,-1,...,323,29525,0,1525,200,323,29525,0,0,0
15321,340000,2,1,2,34,-1,-1,-1,-1,-1,...,28987,13104,59204,42552,30084,29033,13120,59273,1241,0
2313,260000,2,2,1,40,-2,-2,-2,-2,-2,...,2339,1474,1254,217,613,2339,0,1254,431,0
13667,170000,1,1,2,43,-1,-1,-1,-1,-1,...,41467,1028,2643,5750,200,41467,1328,2643,126,0
3120,150000,1,1,1,37,0,0,0,-1,0,...,50123,45987,46555,1510,1850,101123,1650,1700,0,0


In [7]:
# Aplicar transformaciones al conjunto de validación
dataset_validacion_procesado = transformar_datos_credito(dataframe_validacion_original)
dataset_validacion_procesado.sample(5)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
3092,100000,2,1,2,25,-1,-1,-1,-1,-1,...,3195,3732,42605,1259,586,3195,3732,42605,30583,0
2193,180000,2,1,2,27,-1,-1,-1,-1,-1,...,28004,957,0,1812,5347,28004,957,0,455,1
1309,170000,1,2,1,48,-1,-1,-1,0,0,...,780,0,0,390,780,0,0,0,0,0
7962,50000,1,2,2,24,0,0,0,0,0,...,28614,29133,29340,1963,1539,1200,719,43945,1001,0
3235,50000,1,1,2,27,1,-2,-2,-2,-2,...,-2900,-2900,-2900,0,0,0,0,0,0,0


## 4. Separación de Características y Variable Objetivo

División de los datos en características (variables independientes) y variable objetivo (variable a predecir).

In [8]:
# Separación de matrices de características y vectores objetivo para entrenamiento
matriz_caracteristicas_entrenamiento = dataset_entrenamiento_procesado.drop(columns=["default"])
vector_objetivo_entrenamiento = dataset_entrenamiento_procesado["default"]

# Separación de matrices de características y vectores objetivo para validación
matriz_caracteristicas_validacion = dataset_validacion_procesado.drop(columns=["default"])
vector_objetivo_validacion = dataset_validacion_procesado["default"]

## 5. Construcción del Pipeline de Machine Learning

Creación de un pipeline integrado que incluye preprocesamiento, escalado, selección de características y modelado.

In [9]:
# Importación de componentes para construcción del pipeline de ML
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# Identificación de variables categóricas para transformación
columnas_variables_categoricas = ["SEX", "EDUCATION", "MARRIAGE"]

# Configuración del transformador para variables categóricas
transformador_variables_categoricas = ColumnTransformer(
    transformers=[
        ("codificacion_categoricas", OneHotEncoder(handle_unknown="ignore"), columnas_variables_categoricas)
    ],
    remainder='passthrough'  # Preservar variables numéricas sin modificación
)

# Construcción del pipeline de machine learning integrado
pipeline_prediccion_incumplimiento = Pipeline([
    ("transformacion_categoricas", transformador_variables_categoricas),
    ("escalamiento_caracteristicas", MinMaxScaler()),
    ("seleccion_caracteristicas_relevantes", SelectKBest(
        score_func=f_classif, 
        k=10
    )),
    ("modelo_regresion_logistica", LogisticRegression(random_state=42)),
])

## 6. Optimización de Hiperparámetros

Búsqueda sistemática de los mejores hiperparámetros usando validación cruzada y búsqueda en grilla.

In [10]:
# Importación de herramienta para búsqueda exhaustiva de hiperparámetros
from sklearn.model_selection import GridSearchCV

# Definición del espacio de hiperparámetros para optimización
espacio_busqueda_hiperparametros = {
    # Cantidad de características más relevantes a seleccionar
    "seleccion_caracteristicas_relevantes__k": range(1, 11),
    
    # Parámetro de regularización del modelo (inverso de la fuerza)
    "modelo_regresion_logistica__C": [0.001, 0.01, 0.1, 1, 10, 100],
    
    # Tipo de penalización para regularización
    "modelo_regresion_logistica__penalty": ["l1", "l2"],
    
    # Algoritmo de optimización numérica
    "modelo_regresion_logistica__solver": ["liblinear"],
    
    # Límite de iteraciones para convergencia
    "modelo_regresion_logistica__max_iter": [100, 200]
}

# Configuración del optimizador de hiperparámetros con validación cruzada
optimizador_hiperparametros = GridSearchCV(
    estimator=pipeline_prediccion_incumplimiento, 
    param_grid=espacio_busqueda_hiperparametros, 
    cv=10, 
    scoring='balanced_accuracy', 
    n_jobs=-1
)

# Ejecución del proceso de optimización
optimizador_hiperparametros.fit(matriz_caracteristicas_entrenamiento, vector_objetivo_entrenamiento)

c:\Users\david\LAB-10-prediccion-del-default-usando-logreg-DavidZapataN\.venv\lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('transformacion_categoricas',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('codificacion_categoricas',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE'])])),
                                       ('escalamiento_caracteristicas',
                                        MinMaxScaler()),
                                       ('seleccion_caracteristicas_relevantes',
                                        SelectKBest()),
                                       ('modelo_regre...
                                        LogisticRegression(random_state=42))]),
             n_jobs=-1,
             param_grid={'modelo_regresion_logistica__C': [0.001, 0.01, 0.1, 1,
                                                           10, 100],
                         'modelo_regresion_logistica__max_iter': [100, 200],
                         'modelo_regresion_logistica__penalty': ['l1', 'l2'],
                         'modelo_regresion_logistica__solver': ['liblinear'],
                         'seleccion_caracteristicas_relevantes__k': range(1, 11)},
             scoring='balanced_accuracy')

## 7. Persistencia del Modelo Optimizado

Guardado del modelo entrenado y optimizado para uso posterior en producción.

In [11]:
# Importación de librerías para serialización y compresión
import gzip
import pickle

# Configuración del directorio de modelos
directorio_modelos_entrenados = "../files/models/"
os.makedirs(directorio_modelos_entrenados, exist_ok=True)

# Definición de la ruta del modelo (nombre requerido por tests)
ruta_modelo_prediccion_default = os.path.join(directorio_modelos_entrenados, "model.pkl.gz")

# Serialización y compresión del modelo con hiperparámetros óptimos
with gzip.open(ruta_modelo_prediccion_default, "wb") as archivo_modelo:
    pickle.dump(optimizador_hiperparametros, archivo_modelo)

## 8. Evaluación Integral del Modelo

Evaluación completa del rendimiento del modelo usando múltiples métricas de clasificación.

In [12]:
# Importación de métricas para evaluación exhaustiva de modelos de clasificación
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

def evaluar_rendimiento_modelo_clasificacion(modelo_entrenado, matriz_caracteristicas, vector_objetivo_real, identificador_conjunto):
    """
    Ejecuta evaluación completa del rendimiento de un modelo de clasificación binaria.
    
    Args:
        modelo_entrenado: Estimador de sklearn ya entrenado y optimizado
        matriz_caracteristicas: DataFrame con variables independientes
        vector_objetivo_real: Series con valores verdaderos de la variable objetivo
        identificador_conjunto: String identificador ("train" o "test")
        
    Returns:
        tuple: (diccionario_metricas_rendimiento, diccionario_matriz_confusion_detallada)
    """
    # Generación de predicciones usando el modelo optimizado
    predicciones_modelo = modelo_entrenado.predict(matriz_caracteristicas)

    # Cálculo de métricas fundamentales de clasificación
    precision_predictiva = precision_score(vector_objetivo_real, predicciones_modelo)
    exactitud_balanceada = balanced_accuracy_score(vector_objetivo_real, predicciones_modelo)
    sensibilidad_recall = recall_score(vector_objetivo_real, predicciones_modelo)
    puntuacion_f1_combinada = f1_score(vector_objetivo_real, predicciones_modelo)

    # Estructuración de métricas en formato estandarizado
    diccionario_metricas_rendimiento = {
        'type': 'metrics',
        'dataset': identificador_conjunto,
        'precision': round(precision_predictiva, 4),
        'balanced_accuracy': round(exactitud_balanceada, 4),
        'recall': round(sensibilidad_recall, 4),
        'f1_score': round(puntuacion_f1_combinada, 4)
    }

    # Generación de matriz de confusión para análisis detallado
    matriz_confusion_resultados = confusion_matrix(vector_objetivo_real, predicciones_modelo)
    
    # Estructuración de matriz de confusión en formato interpretable
    diccionario_matriz_confusion_detallada = {
        'type': 'cm_matrix',
        'dataset': identificador_conjunto,
        'true_0': {
            'predicted_0': int(matriz_confusion_resultados[0, 0]),  # Verdaderos Negativos
            'predicted_1': int(matriz_confusion_resultados[0, 1])   # Falsos Positivos
        },
        'true_1': {
            'predicted_0': int(matriz_confusion_resultados[1, 0]),  # Falsos Negativos
            'predicted_1': int(matriz_confusion_resultados[1, 1])   # Verdaderos Positivos
        }
    }

    return diccionario_metricas_rendimiento, diccionario_matriz_confusion_detallada

# Evaluación del modelo en conjunto de entrenamiento (usando nombres esperados por tests)
metricas_conjunto_entrenamiento, confusion_conjunto_entrenamiento = evaluar_rendimiento_modelo_clasificacion(
    optimizador_hiperparametros, matriz_caracteristicas_entrenamiento, vector_objetivo_entrenamiento, "train"
)

# Evaluación del modelo en conjunto de validación (usando nombres esperados por tests)
metricas_conjunto_validacion, confusion_conjunto_validacion = evaluar_rendimiento_modelo_clasificacion(
    optimizador_hiperparametros, matriz_caracteristicas_validacion, vector_objetivo_validacion, "test"
)

# Consolidación de todos los resultados de evaluación
resultados_evaluacion_completos = [
    metricas_conjunto_entrenamiento, 
    metricas_conjunto_validacion, 
    confusion_conjunto_entrenamiento, 
    confusion_conjunto_validacion
]

## 9. Exportación de Resultados de Evaluación

Persistencia de métricas y matrices de confusión en formato JSON para análisis posterior y documentación.

In [13]:
# Importación de librería para serialización de datos estructurados
import json

# Asegurar existencia del directorio de resultados
os.makedirs(RUTA_RESULTADOS_SALIDA, exist_ok=True)

# Definición de archivo de salida para métricas (nombre requerido por tests)
archivo_resultados_evaluacion = os.path.join(RUTA_RESULTADOS_SALIDA, "metrics.json")

# Exportación de resultados en formato JSON Lines para procesamiento posterior
with open(archivo_resultados_evaluacion, "w", encoding="utf-8") as archivo_metricas:
    for resultado_individual in resultados_evaluacion_completos:
        archivo_metricas.write(json.dumps(resultado_individual) + "\n")